In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, auc
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Input, Multiply, TimeDistributed, Softmax, Lambda, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from xgboost import XGBClassifier
from tensorflow.keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import copy           

def print_results(outcome_variable, lr_prob, proportion):
        ### Training data:
        # Load data from a CSV file
        df = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_lr.csv')
        df_mask = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_mask.csv')
        df_delta = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_delta.csv')

        Africa=["Zambia", "Kenya", "Ghana"]
        Asia=["Pakistan", "India-CMC", "India-SAS"]
    
       # Drop rows with NaN in the target column before any operations
        df = df.dropna(subset=[outcome_variable])
        df = df[df['SITE'].isin(Asia)]
        df = df.reset_index(drop=True)
    
        df_mask = df_mask.dropna(subset=[outcome_variable])
        df_mask = df_mask[df_mask['SITE'].isin(Asia)]
        df_mask = df_mask.reset_index(drop=True)
    
        df_delta = df_delta.dropna(subset=[outcome_variable])
        df_delta = df_delta[df_delta['SITE'].isin(Asia)]
        df_delta = df_delta.reset_index(drop=True)
        
        features_categorical = ["TYPE_VISIT",
                    "STILLBIRTH_IND","PRETERM_IND","CESARIAN_IND","NUM_MISCARRIAGE_ind",
                    "DEPR_EVER",
                    "PAID_WORK","GPARITY_2","GPARITY_1","WEALTH_QUINT_1","WEALTH_QUINT_2","WEALTH_QUINT_3","WEALTH_QUINT_4","SCHOOL_MORE10","WATER_IMPROVED","TOILET_IMPROVED","STOVE_FUEL","HH_SMOKE","SMOKE","CHEW_TOBACCO","CHEW_BETELNUT",
                    "BMI_LEVEL_ENROLL_underweight","BMI_LEVEL_ENROLL_overweight","BMI_LEVEL_ENROLL_obese",
                    "GWG_ADEQUACY_inadequate","GWG_ADEQUACY_excessive",
                    "MAT_CES_ANY","BIRTH_FACILITY",
                    "HTN","DIAB_OVERT_ANY","HIV_POSITIVE","TB_SYMP_POSITIVE","MAL_POSITIVE","HBV_POSITIVE_ENROLL","HCV_POSITIVE_ENROLL","STI_POSITIVE",
                    "GES_HTN","DIAB_GEST_ANY",
                    "INF_SEX","INF_HYPERBILI_NICE","BREASTFED",
                    "NUM_FETUS_2","NUM_FETUS_3","INF_ANOMALY","PREVIA",
                    "INF_PSBI_ANY",
                    "M08_ANEMIA"]
        
        features_continuous = ["MAT_AGE","MUAC_ENROLL","BMI_ENROLL","GESTAGEBIRTH_ANY","BWEIGHT_ANY","CROWDING_IND",
                              "M01_AFI_PERES",
                              "M08_IRON_TOT_UGDL_LBORRES", "M08_FERRITIN_LBORRES","M08_IRON_HEP_LBORRES","M08_CBC_HB_LBORRES"]
        
        features_static = ["STILLBIRTH_IND","PRETERM_IND","CESARIAN_IND","NUM_MISCARRIAGE_ind",
                    "DEPR_EVER",
                    "PAID_WORK","GPARITY_2","GPARITY_1","WEALTH_QUINT_1","WEALTH_QUINT_2","WEALTH_QUINT_3","WEALTH_QUINT_4","SCHOOL_MORE10","WATER_IMPROVED","TOILET_IMPROVED","STOVE_FUEL","HH_SMOKE","SMOKE","CHEW_TOBACCO","CHEW_BETELNUT","CROWDING_IND",
                    "BMI_LEVEL_ENROLL_underweight","BMI_LEVEL_ENROLL_overweight","BMI_LEVEL_ENROLL_obese",
                    "GWG_ADEQUACY_inadequate","GWG_ADEQUACY_excessive",
                    "MAT_CES_ANY","BIRTH_FACILITY",
                    "HTN","DIAB_OVERT_ANY","HIV_POSITIVE","TB_SYMP_POSITIVE","MAL_POSITIVE","HBV_POSITIVE_ENROLL","HCV_POSITIVE_ENROLL","STI_POSITIVE",
                    "GES_HTN","DIAB_GEST_ANY",
                    "INF_SEX","INF_HYPERBILI_NICE","BREASTFED","GESTAGEBIRTH_ANY","BWEIGHT_ANY",
                    "INF_ANOMALY",
                    "INF_PSBI_ANY",
                    "NUM_FETUS_2","NUM_FETUS_3","MAT_AGE","MUAC_ENROLL",
                    "BMI_ENROLL"]
        
        features_temporal = ["TYPE_VISIT",
                             "M01_AFI_PERES",
                             "PREVIA",
                             "M08_IRON_TOT_UGDL_LBORRES","M08_FERRITIN_LBORRES","M08_IRON_HEP_LBORRES","M08_CBC_HB_LBORRES","M08_ANEMIA"]

        features_to_remove_by_outcome = {
        "STILLBIRTH_SIGNS_LIFE": ["INF_HYPERBILI_NICE","BREASTFED","BWEIGHT_ANY","INF_PSBI_ANY"],
        "PRETERMBIRTH_LT37": ["INF_HYPERBILI_NICE","BREASTFED","GESTAGEBIRTH_ANY","BWEIGHT_ANY","INF_PSBI_ANY","MAT_CES_ANY","BIRTH_FACILITY"],
        "LBW2500_ANY": ["INF_HYPERBILI_NICE","BREASTFED","BWEIGHT_ANY","INF_PSBI_ANY","MAT_CES_ANY","BIRTH_FACILITY"],
        "INF_PSBI_OUTCOME": ["INF_HYPERBILI_NICE","INF_PSBI_ANY","PREVIA"],
        "SVN": ["INF_SEX","INF_HYPERBILI_NICE","GESTAGEBIRTH_ANY","BWEIGHT_ANY","INF_PSBI_ANY","BREASTFED","MAT_CES_ANY","BIRTH_FACILITY"],
        "NEARMISS": ["INF_SEX","INF_HYPERBILI_NICE","BREASTFED","GESTAGEBIRTH_ANY","BWEIGHT_ANY","INF_PSBI_ANY"]
        }
    
        features_to_remove = features_to_remove_by_outcome.get(outcome_variable, [])
        
        # Remove features from the lists
        features_categorical = [f for f in features_categorical if f not in features_to_remove]
        features_continuous = [f for f in features_continuous if f not in features_to_remove]
        features_static = [f for f in features_static if f not in features_to_remove]
        features_temporal = [f for f in features_temporal if f not in features_to_remove]
        
        # Separate categorical and continuous features
        df_categorical = df[features_categorical]
        df_continuous = df[features_continuous]
        
        # Rescale continuous features to (0, 1) range
        lower_q = 0.02   # 2nd percentile
        upper_q = 0.98   # 98th percentile
        
        df_continuous_capped = df_continuous.copy()
        
        for col in features_continuous:
            lower = df_continuous[col].quantile(lower_q)
            upper = df_continuous[col].quantile(upper_q)
            df_continuous_capped[col] = df_continuous[col].clip(lower, upper)
        
        scaler = MinMaxScaler()
        df_continuous_scaled = pd.DataFrame(
            scaler.fit_transform(df_continuous_capped),
            columns=features_continuous,
            index=df.index
        )
        
        # Concatenate scaled continuous features with categorical features
        X = pd.concat([df_categorical, df_continuous_scaled], axis=1)
        
        ##Static and temporal data
        X_static = X[features_static]
        
        X_temporal = X[features_temporal]
        X_temporal_delta = df_delta[features_temporal]
        X_temporal_mask = df_mask[features_temporal]
        
        y = df[outcome_variable]
        
        # One-hot encoding for non-binary categorical features in X
        X = pd.get_dummies(X, columns=['TYPE_VISIT'], dummy_na=False)
        X_temporal = pd.get_dummies(X_temporal, columns=['TYPE_VISIT'], dummy_na=False)
        X_temporal_delta = pd.get_dummies(X_temporal_delta, columns=['TYPE_VISIT'], dummy_na=False)
        X_temporal_mask = pd.get_dummies(X_temporal_mask, columns=['TYPE_VISIT'], dummy_na=False)
        
        # Preprocess for Keras models (3D matrix)
        n_samples = X_temporal.shape[0] // 5 
        X_static_keras = X_static.iloc[::5].values.astype('float32')
        
        X_temporal_keras = X_temporal.values.reshape(n_samples, 5, X_temporal.shape[1]).astype('float32')
        X_temporal_delta_keras = X_temporal_delta.values.reshape(n_samples, 5, X_temporal_delta.shape[1]).astype('float32')
        X_temporal_mask_keras = X_temporal_mask.values.reshape(n_samples, 5, X_temporal_mask.shape[1]).astype('float32')
        
        X_keras = X.values.reshape(n_samples, 5, X.shape[1]).astype('float32')
        
        # Align y for Keras models
        y_keras = y.iloc[::5].values.astype('float32')

        ### Target data:
        # Load data from a CSV file
        df_target = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_Africa_train.csv')
        df_target_mask = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_mask_Africa_train.csv')
        df_target_delta = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_delta_Africa_train.csv')
        
       # Drop rows with NaN in the target column before any operations
        df_target = df_target.dropna(subset=[outcome_variable])
        df_target = df_target.reset_index(drop=True)
    
        df_target_mask = df_target_mask.dropna(subset=[outcome_variable])
        df_target_mask = df_target_mask.reset_index(drop=True)
    
        df_target_delta = df_target_delta.dropna(subset=[outcome_variable])
        df_target_delta = df_target_delta.reset_index(drop=True)
    
        # Separate categorical and continuous features
        df_target_categorical = df_target[features_categorical]
        df_target_continuous = df_target[features_continuous]
        
        # get the effective train bounds (same as the ones used in training)
        cap_bounds = {col: (df_continuous[col].quantile(lower_q), df_continuous[col].quantile(upper_q))
                      for col in features_continuous}
        
        df_target_continuous_capped = df_target_continuous.copy()
        for col in features_continuous:
            lower, upper = cap_bounds[col]
            df_target_continuous_capped[col] = df_target_continuous[col].clip(lower, upper)
        
        df_target_continuous_scaled = pd.DataFrame(
            scaler.transform(df_target_continuous_capped),
            columns=features_continuous,
            index=df_target_continuous.index
        )
        
        # Concatenate scaled continuous features with categorical features
        X_target = pd.concat([df_target_categorical, df_target_continuous_scaled], axis=1)
          
        ##Static and temporal data
        X_target_static = X_target[features_static]
        
        X_target_temporal = X_target[features_temporal]
        X_target_temporal_delta = df_target_delta[features_temporal]
        X_target_temporal_mask = df_target_mask[features_temporal]

        y_target_soft = df_target[lr_prob]
        y_target = df_target[outcome_variable]
        threshold = (df_target[outcome_variable] == 1).mean()
        
        # One-hot encoding for non-binary categorical features in X
        X_target = pd.get_dummies(X_target, columns=['TYPE_VISIT'], dummy_na=False)
        X_target_temporal = pd.get_dummies(X_target_temporal, columns=['TYPE_VISIT'], dummy_na=False)
        X_target_temporal_delta = pd.get_dummies(X_target_temporal_delta, columns=['TYPE_VISIT'], dummy_na=False)
        X_target_temporal_mask = pd.get_dummies(X_target_temporal_mask, columns=['TYPE_VISIT'], dummy_na=False)
        
        # Preprocess for Keras models (3D matrix)
        n_target_samples = X_target_temporal.shape[0] // 5 
        X_target_static_keras = X_target_static.iloc[::5].values.astype('float32')
        
        X_target_temporal_keras = X_target_temporal.values.reshape(n_target_samples, 5, X_target_temporal.shape[1]).astype('float32')
        X_target_temporal_delta_keras = X_target_temporal_delta.values.reshape(n_target_samples, 5, X_target_temporal_delta.shape[1]).astype('float32')
        X_target_temporal_mask_keras = X_target_temporal_mask.values.reshape(n_target_samples, 5, X_target_temporal_mask.shape[1]).astype('float32')
        
        X_target_keras = X_target.values.reshape(n_target_samples, 5, X_target.shape[1]).astype('float32')
        
        # Align y for Keras models
        y_target_soft_keras = y_target_soft.iloc[::5].values.astype('float32')

        ### Test data:
        # Load data from a CSV file
        df_test = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_Africa.csv')
        df_test_mask = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_mask_Africa.csv')
        df_test_delta = pd.read_csv('D:/Users/yipeng_wei/Documents/dl data/2025-10-31/df_dl_continuous_delta_Africa.csv')
        
        # Drop rows with NaN in the test column before any operations
        df_test = df_test.dropna(subset=[outcome_variable])
        df_test = df_test.reset_index(drop=True)
    
        df_test_mask = df_test_mask.dropna(subset=[outcome_variable])
        df_test_mask = df_test_mask.reset_index(drop=True)
    
        df_test_delta = df_test_delta.dropna(subset=[outcome_variable])
        df_test_delta = df_test_delta.reset_index(drop=True)
    
        # Separate categorical and continuous features
        df_test_categorical = df_test[features_categorical]
        df_test_continuous = df_test[features_continuous]
        
        # get the effective train bounds (same as the ones used in training)
        cap_bounds = {col: (df_continuous[col].quantile(lower_q), df_continuous[col].quantile(upper_q))
                      for col in features_continuous}
        
        df_test_continuous_capped = df_test_continuous.copy()
        for col in features_continuous:
            lower, upper = cap_bounds[col]
            df_test_continuous_capped[col] = df_test_continuous[col].clip(lower, upper)
        
        df_test_continuous_scaled = pd.DataFrame(
            scaler.transform(df_test_continuous_capped),
            columns=features_continuous,
            index=df_test_continuous.index
        )
        
        # Concatenate scaled continuous features with categorical features
        X_test = pd.concat([df_test_categorical, df_test_continuous_scaled], axis=1)
          
        ##Static and temporal data
        X_test_static = X_test[features_static]
        
        X_test_temporal = X_test[features_temporal]
        X_test_temporal_delta = df_test_delta[features_temporal]
        X_test_temporal_mask = df_test_mask[features_temporal]

        y_test = df_test[outcome_variable]
        threshold = (df_test[outcome_variable] == 1).mean()
        
        # One-hot encoding for non-binary categorical features in X
        X_test = pd.get_dummies(X_test, columns=['TYPE_VISIT'], dummy_na=False)
        X_test_temporal = pd.get_dummies(X_test_temporal, columns=['TYPE_VISIT'], dummy_na=False)
        X_test_temporal_delta = pd.get_dummies(X_test_temporal_delta, columns=['TYPE_VISIT'], dummy_na=False)
        X_test_temporal_mask = pd.get_dummies(X_test_temporal_mask, columns=['TYPE_VISIT'], dummy_na=False)
        
        # Preprocess for Keras models (3D matrix)
        n_test_samples = X_test_temporal.shape[0] // 5 
        X_test_static_keras = X_test_static.iloc[::5].values.astype('float32')
        
        X_test_temporal_keras = X_test_temporal.values.reshape(n_test_samples, 5, X_test_temporal.shape[1]).astype('float32')
        X_test_temporal_delta_keras = X_test_temporal_delta.values.reshape(n_test_samples, 5, X_test_temporal_delta.shape[1]).astype('float32')
        X_test_temporal_mask_keras = X_test_temporal_mask.values.reshape(n_test_samples, 5, X_test_temporal_mask.shape[1]).astype('float32')
        
        X_test_keras = X_test.values.reshape(n_test_samples, 5, X_test.shape[1]).astype('float32')
        
        # Align y for Keras models
        y_test_keras = y_test.iloc[::5].values.astype('float32')

        # Set random seed for reproducibility
        np.random.seed(42)
        
        # Total number of samples
        n_select = X_target_static_keras.shape[0]
        
        # Randomly select 10% of indices
        selected_indices = np.random.choice(n_select, size=int(n_select * proportion), replace=False)
    
        # Sort indices (optional, for consistent ordering)
        selected_indices = np.sort(selected_indices)
        
        # Subset all arrays using the same indices
        X_target_static_keras_sub = X_target_static_keras[selected_indices]
        X_target_temporal_keras_sub = X_target_temporal_keras[selected_indices]
        X_target_temporal_delta_keras_sub = X_target_temporal_delta_keras[selected_indices]
        X_target_temporal_mask_keras_sub = X_target_temporal_mask_keras[selected_indices]
        y_target_soft_keras_sub = y_target_soft_keras[selected_indices]

        #### GRUD With Static Attention
        class GRUDWithStaticAttention(nn.Module):
            def __init__(self, input_size, hidden_size, output_size, mean_values, static_input_size, static_embedding_size, dropout_rate=0.5):
                """
                GRU-D model with attention mechanism generated from static data.
                
                Args:
                    input_size (int): Number of input features for temporal data.
                    hidden_size (int): Number of hidden units in GRU.
                    output_size (int): Number of output classes (for classification tasks).
                    mean_values (torch.Tensor): Empirical mean values for each input feature.
                    static_input_size (int): Number of input features for static data.
                    static_embedding_size (int): Size of the embedding for static data.
                """
                super(GRUDWithStaticAttention, self).__init__()
                
                # Temporal (GRU-D) parameters
                self.input_size = input_size
                self.hidden_size = hidden_size
                self.output_size = output_size
                
                self.mean_values = mean_values
                
                self.gamma_x = nn.Linear(input_size, input_size)
                
                # GRU gate parameters (temporal part) with combined input, hidden state, and mask
                self.zl = nn.Linear(input_size + hidden_size + input_size, hidden_size)  # Update gate
                self.rl = nn.Linear(input_size + hidden_size + input_size, hidden_size)  # Reset gate
                self.hl = nn.Linear(input_size + hidden_size + input_size, hidden_size)  # Candidate hidden state
        
                # Embedding or processing static data
                self.static_fc = nn.Linear(static_input_size, static_embedding_size)
        
                # Attention mechanism based on static data
                self.attention = nn.Linear(static_embedding_size, hidden_size)
        
                # Fully connected output layer
                self.dropout = nn.Dropout(dropout_rate)
                self.concat_fc = nn.Linear(hidden_size + static_embedding_size, output_size)

        
            def forward(self, x, m, delta, static_data):
                """
                Forward pass of GRU-D with static data attention.
                
                Args:
                    x (torch.Tensor): Temporal input data [batch_size, seq_len, input_size].
                    m (torch.Tensor): Masking vector [batch_size, seq_len, input_size].
                    delta (torch.Tensor): Time intervals [batch_size, seq_len, input_size].
                    static_data (torch.Tensor): Static data input [batch_size, static_input_size].
                """
                batch_size, seq_len, _ = x.size()
                
                # Process static data to generate attention weights
                static_embed = torch.relu(self.static_fc(static_data))  # [batch_size, static_embedding_size]
                
                # Generate attention weights from static data
                attention_weights = torch.sigmoid(self.attention(static_embed))  # [batch_size, hidden_size]
                
                # Initialize hidden state
                h = torch.zeros(batch_size, self.hidden_size).to(x.device)
                
                for t in range(seq_len):
                    # Corrected input decay mechanism
                    gamma_x_t = torch.exp(-F.relu(self.gamma_x(delta[:, t, :])))
                    x_t_hat = m[:, t, :] * x[:, t, :] + (1 - m[:, t, :]) * ((1 - gamma_x_t) * self.mean_values + gamma_x_t * x[:, t, :])
                    
                    # Concatenate input, hidden state, and mask
                    combined = torch.cat([x_t_hat, h, m[:, t, :]], dim=-1)
        
                    # GRU gates
                    r_t = torch.sigmoid(self.rl(combined))  # Reset gate
                    z_t = torch.sigmoid(self.zl(combined))  # Update gate
                    h_tilde = torch.tanh(self.hl(torch.cat([x_t_hat, r_t * h, m[:, t, :]], dim=-1)))  # Candidate hidden state
        
                    # Update hidden state
                    h = (1 - z_t) * h + z_t * h_tilde
        
                # Apply attention weights to the final hidden state
                h_weighted = attention_weights * h  # [batch_size, hidden_size]

                h_concat = self.dropout(torch.cat([h_weighted, static_embed], dim=-1))  # [B, hidden + static_embedding_size]
            
                output = torch.sigmoid(self.concat_fc(h_concat)) 
                return output
        
        # Training Parameters
        input_size = X_temporal_keras.shape[2]  # Temporal input features
        hidden_size = 32  # GRU hidden state size
        output_size = 1  # Binary classification
        static_input_size = X_static_keras.shape[1]  # Static input features
        static_embedding_size = 32  # Embedding size for static features
        mean_values = torch.tensor(np.nanmean(X_temporal_keras, axis=(0, 1)), dtype=torch.float32)  # Temporal feature means

        # Convert data to PyTorch tensors
        x = torch.tensor(X_temporal_keras, dtype=torch.float32)  # Temporal input
        mask = torch.tensor(X_temporal_mask_keras, dtype=torch.float32)  # Masking vector
        delta = torch.tensor(X_temporal_delta_keras, dtype=torch.float32)  # Time intervals
        static_data = torch.tensor(X_static_keras, dtype=torch.float32)  # Static input
        y = torch.tensor(y_keras, dtype=torch.float32)  # Binary labels
        
        def GRUD_Static_Model_trained(model_class, x, mask, delta, static_data, y, n_epochs, learning_rate):
            model = model_class(
                input_size=x.shape[2],
                hidden_size=hidden_size,
                output_size=output_size,
                mean_values=mean_values,
                static_input_size=static_input_size,
                static_embedding_size=static_embedding_size,
            )
        
            criterion = nn.BCELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
            for epoch in range(n_epochs):
                model.train()
                optimizer.zero_grad()
                outputs = model(x, mask, delta, static_data)
                loss = criterion(outputs.squeeze(), y.squeeze())
                loss.backward()
                optimizer.step()
        
            model.eval()
            return model

        trained_model = GRUD_Static_Model_trained(
            GRUDWithStaticAttention,
            x,
            mask,
            delta,
            static_data,
            y,
            n_epochs=100,
            learning_rate=0.01
        )
        
        # Convert test data to torch tensors
        x_test = torch.tensor(X_test_temporal_keras, dtype=torch.float32)
        mask_test = torch.tensor(X_test_temporal_mask_keras, dtype=torch.float32)
        delta_test = torch.tensor(X_test_temporal_delta_keras, dtype=torch.float32)
        static_test = torch.tensor(X_test_static_keras, dtype=torch.float32)
        y_test_true = y_test_keras.astype(int)  # Convert to 0/1 labels

        x_target_sub = torch.tensor(X_target_temporal_keras_sub, dtype=torch.float32)
        mask_target_sub = torch.tensor(X_target_temporal_mask_keras_sub, dtype=torch.float32)
        delta_target_sub = torch.tensor(X_target_temporal_delta_keras_sub, dtype=torch.float32)
        static_target_sub = torch.tensor(X_target_static_keras_sub, dtype=torch.float32)
        y_target_soft_sub = torch.tensor(y_target_soft_keras_sub, dtype=torch.float32)

        def fine_tune_model_freeze(
            model, x, mask, delta, static_data, y_soft,
            n_epochs, learning_rate,
            patience, min_delta, restore_best=True,
            verbose=False
        ):
            # Freeze all layers
            for param in model.parameters():
                param.requires_grad = False
        
            # Unfreeze attention and final output layers
            for name, param in model.named_parameters():
                if ('attention' in name) or ('fc' in name):
                    param.requires_grad = True
        
            model.train()
            optimizer = torch.optim.Adam(
                (p for p in model.parameters() if p.requires_grad),
                lr=learning_rate
            )
            criterion = nn.BCELoss()
        
            best_loss = float("inf")
            best_state = None
            epochs_no_improve = 0
        
            for epoch in range(n_epochs):
                optimizer.zero_grad()
        
                outputs = model(x, mask, delta, static_data).squeeze()
                loss = criterion(outputs, y_soft)
        
                loss.backward()
                optimizer.step()
        
                current_loss = float(loss.item())
        
                # Early stopping check
                improved = (best_loss - current_loss) > min_delta
                if improved:
                    best_loss = current_loss
                    epochs_no_improve = 0
                    if restore_best:
                        best_state = copy.deepcopy(model.state_dict())
                else:
                    epochs_no_improve += 1
        
                if epochs_no_improve >= patience:
                    if verbose:
                        print(f"Early stopping at epoch {epoch+1}")
                    break
        
            if restore_best and best_state is not None:
                model.load_state_dict(best_state)
        
            model.eval()
            return model
            
        fine_tuned_model = fine_tune_model_freeze(
            trained_model,
            x_target_sub, mask_target_sub, delta_target_sub, static_target_sub,
            y_target_soft_sub, 
            n_epochs=1000,          # max epochs (upper bound)
            learning_rate=0.01,
            patience=10,           # stop if no improvement for 5 epochs
            min_delta=1e-4,       # minimum loss improvement
            restore_best=True,    # reload best weights
            verbose=True          # optional logging
        )

       # Evaluate fine-tuned model on test set
        fine_tuned_model.eval()
        with torch.no_grad():
            probs = fine_tuned_model(x_test, mask_test, delta_test, static_test).squeeze().cpu().numpy()
        
        def evaluate_predictions(y_true, y_prob, threshold=threshold):
            y_pred = (y_prob > threshold).astype(int)
            return {
                "Outcome": outcome,
                "Precision": precision_score(y_true, y_pred, zero_division=0).round(3),
                "Recall": recall_score(y_true, y_pred, zero_division=0).round(3),
                "F1 Score": f1_score(y_true, y_pred, zero_division=0).round(3),
                "ROC-AUC": roc_auc_score(y_true, y_prob).round(3),
                "threshold": threshold.round(3)
            }
        
        # Evaluate and print results
        metrics = evaluate_predictions(y_test_true, probs, threshold)
        
        return metrics


In [2]:
outcomes = ["STILLBIRTH_SIGNS_LIFE","LBW2500_ANY", "SVN", "NEARMISS", "NEO_DTH"]
probs = ['stillbirth.prob', 'lbw.prob', 'svn.prob', 'nearmiss.prob','neo_dth.prob']

# Function to process results for an outcome
def process_results(outcome,prob):
    # Run the function and capture the returned values
    results_summary= print_results(outcome,prob,0.05)
    print(results_summary)
    
# Run the function for each outcome
for outcome, prob in zip(outcomes, probs):
    process_results(outcome,prob)

Early stopping at epoch 67
{'Outcome': 'STILLBIRTH_SIGNS_LIFE', 'Precision': 0.173, 'Recall': 0.703, 'F1 Score': 0.278, 'ROC-AUC': 0.859, 'threshold': 0.026}
Early stopping at epoch 45
{'Outcome': 'LBW2500_ANY', 'Precision': 0.464, 'Recall': 0.575, 'F1 Score': 0.514, 'ROC-AUC': 0.835, 'threshold': 0.142}
Early stopping at epoch 86
{'Outcome': 'SVN', 'Precision': 0.317, 'Recall': 0.614, 'F1 Score': 0.419, 'ROC-AUC': 0.655, 'threshold': 0.239}
Early stopping at epoch 79
{'Outcome': 'NEARMISS', 'Precision': 0.114, 'Recall': 0.53, 'F1 Score': 0.188, 'ROC-AUC': 0.606, 'threshold': 0.083}
Early stopping at epoch 46
{'Outcome': 'NEO_DTH', 'Precision': 0.044, 'Recall': 0.9, 'F1 Score': 0.084, 'ROC-AUC': 0.92, 'threshold': 0.006}


In [3]:
outcomes = ["STILLBIRTH_SIGNS_LIFE","LBW2500_ANY", "SVN", "NEARMISS", "NEO_DTH"]
probs = ['stillbirth.prob', 'lbw.prob', 'svn.prob', 'nearmiss.prob','neo_dth.prob']

# Function to process results for an outcome
def process_results(outcome,prob):
    # Run the function and capture the returned values
    results_summary= print_results(outcome,prob,0.1)
    print(results_summary)
    
# Run the function for each outcome
for outcome, prob in zip(outcomes, probs):
    process_results(outcome,prob)

Early stopping at epoch 83
{'Outcome': 'STILLBIRTH_SIGNS_LIFE', 'Precision': 0.204, 'Recall': 0.784, 'F1 Score': 0.324, 'ROC-AUC': 0.906, 'threshold': 0.026}
Early stopping at epoch 82
{'Outcome': 'LBW2500_ANY', 'Precision': 0.425, 'Recall': 0.684, 'F1 Score': 0.524, 'ROC-AUC': 0.842, 'threshold': 0.142}
Early stopping at epoch 78
{'Outcome': 'SVN', 'Precision': 0.307, 'Recall': 0.687, 'F1 Score': 0.424, 'ROC-AUC': 0.655, 'threshold': 0.239}
Early stopping at epoch 93
{'Outcome': 'NEARMISS', 'Precision': 0.112, 'Recall': 0.515, 'F1 Score': 0.184, 'ROC-AUC': 0.603, 'threshold': 0.083}
Early stopping at epoch 42
{'Outcome': 'NEO_DTH', 'Precision': 0.036, 'Recall': 0.7, 'F1 Score': 0.069, 'ROC-AUC': 0.919, 'threshold': 0.006}


In [4]:
outcomes = ["STILLBIRTH_SIGNS_LIFE","LBW2500_ANY", "SVN", "NEARMISS", "NEO_DTH"]
probs = ['stillbirth.prob', 'lbw.prob', 'svn.prob', 'nearmiss.prob','neo_dth.prob']

# Function to process results for an outcome
def process_results(outcome,prob):
    # Run the function and capture the returned values
    results_summary= print_results(outcome,prob,0.2)
    print(results_summary)
    
# Run the function for each outcome
for outcome, prob in zip(outcomes, probs):
    process_results(outcome,prob)

Early stopping at epoch 74
{'Outcome': 'STILLBIRTH_SIGNS_LIFE', 'Precision': 0.265, 'Recall': 0.811, 'F1 Score': 0.4, 'ROC-AUC': 0.923, 'threshold': 0.026}
Early stopping at epoch 70
{'Outcome': 'LBW2500_ANY', 'Precision': 0.431, 'Recall': 0.646, 'F1 Score': 0.517, 'ROC-AUC': 0.835, 'threshold': 0.142}
Early stopping at epoch 68
{'Outcome': 'SVN', 'Precision': 0.302, 'Recall': 0.661, 'F1 Score': 0.415, 'ROC-AUC': 0.652, 'threshold': 0.239}
Early stopping at epoch 121
{'Outcome': 'NEARMISS', 'Precision': 0.118, 'Recall': 0.485, 'F1 Score': 0.19, 'ROC-AUC': 0.598, 'threshold': 0.083}
Early stopping at epoch 42
{'Outcome': 'NEO_DTH', 'Precision': 0.035, 'Recall': 0.7, 'F1 Score': 0.066, 'ROC-AUC': 0.919, 'threshold': 0.006}


In [7]:
outcomes = ["STILLBIRTH_SIGNS_LIFE","LBW2500_ANY", "SVN", "NEARMISS", "NEO_DTH"]
probs = ['stillbirth.prob', 'lbw.prob', 'svn.prob', 'nearmiss.prob','neo_dth.prob']

# Function to process results for an outcome
def process_results(outcome,prob):
    # Run the function and capture the returned values
    results_summary= print_results(outcome,prob,0.5)
    print(results_summary)
    
# Run the function for each outcome
for outcome, prob in zip(outcomes, probs):
    process_results(outcome,prob)

Early stopping at epoch 110
{'Outcome': 'STILLBIRTH_SIGNS_LIFE', 'Precision': 0.265, 'Recall': 0.811, 'F1 Score': 0.4, 'ROC-AUC': 0.936, 'threshold': 0.026}
Early stopping at epoch 132
{'Outcome': 'LBW2500_ANY', 'Precision': 0.386, 'Recall': 0.726, 'F1 Score': 0.504, 'ROC-AUC': 0.843, 'threshold': 0.142}
Early stopping at epoch 158
{'Outcome': 'SVN', 'Precision': 0.307, 'Recall': 0.69, 'F1 Score': 0.425, 'ROC-AUC': 0.659, 'threshold': 0.239}
Early stopping at epoch 128
{'Outcome': 'NEARMISS', 'Precision': 0.108, 'Recall': 0.53, 'F1 Score': 0.18, 'ROC-AUC': 0.596, 'threshold': 0.083}
Early stopping at epoch 76
{'Outcome': 'NEO_DTH', 'Precision': 0.041, 'Recall': 0.8, 'F1 Score': 0.078, 'ROC-AUC': 0.909, 'threshold': 0.006}


In [8]:
outcomes = ["STILLBIRTH_SIGNS_LIFE","LBW2500_ANY", "SVN", "NEARMISS", "NEO_DTH"]
probs = ['stillbirth.prob', 'lbw.prob', 'svn.prob', 'nearmiss.prob','neo_dth.prob']

# Function to process results for an outcome
def process_results(outcome,prob):
    # Run the function and capture the returned values
    results_summary= print_results(outcome,prob,1)
    print(results_summary)
    
# Run the function for each outcome
for outcome, prob in zip(outcomes, probs):
    process_results(outcome,prob)

Early stopping at epoch 82
{'Outcome': 'STILLBIRTH_SIGNS_LIFE', 'Precision': 0.252, 'Recall': 0.784, 'F1 Score': 0.382, 'ROC-AUC': 0.938, 'threshold': 0.026}
Early stopping at epoch 82
{'Outcome': 'LBW2500_ANY', 'Precision': 0.43, 'Recall': 0.665, 'F1 Score': 0.522, 'ROC-AUC': 0.84, 'threshold': 0.142}
Early stopping at epoch 120
{'Outcome': 'SVN', 'Precision': 0.31, 'Recall': 0.693, 'F1 Score': 0.429, 'ROC-AUC': 0.658, 'threshold': 0.239}
Early stopping at epoch 143
{'Outcome': 'NEARMISS', 'Precision': 0.111, 'Recall': 0.515, 'F1 Score': 0.183, 'ROC-AUC': 0.602, 'threshold': 0.083}
Early stopping at epoch 113
{'Outcome': 'NEO_DTH', 'Precision': 0.037, 'Recall': 0.8, 'F1 Score': 0.07, 'ROC-AUC': 0.887, 'threshold': 0.006}
